In [45]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate
from langchain.memory import ConversationSummaryBufferMemory
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.schema.runnable import RunnablePassthrough
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder

llm = ChatOpenAI(
    temperature=0.1, streaming=True, callbacks=[StreamingStdOutCallbackHandler()]
)

memory = ConversationSummaryBufferMemory(
    llm=llm, max_token_limit=120, return_messages=True
)

# Few-shot 예제 데이터 (학습용)
examples = [
    {"question": "The Godfather?", "answer": "👨‍👨‍👦🔫🍝"},
    {"question": "TopGun?", "answer": "🛩️👨‍✈️🔥"},
]

example_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=ChatPromptTemplate.from_messages(
        [("human", "{question}"), ("ai", "{answer}")]
    ),
    examples=examples,
)

# final_prompt = ChatPromptTemplate.from_messages(
#     [
#         (
#             "system",
#             "You are a movie expert. If a human tells you the title of the movie, you have to respond with 3 emoticons."
#             "However, for the first movie question in a conversation, you must answer with the movie title instead of emojis. "
#             "The movies of the bleow examples shoulb not be provided to users in any answers."
#             "Always refer to the history to determine if this is the first movie question.",
#         ),
#         example_prompt,
#         ("system", "history:\n{history}"),  # 대화 이력을 시스템에 포함
#         ("human", "{question}"),
#     ]
# )


final_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a movie expert. You know every movie. If a human tells you the title of the movie, you have to respond with 3 emoticons.",
        ),
        example_prompt,
        (
            "system",
            "The above examples should not be provided to the user. The user can only be provided with the conversation record below. Please provide the information to the user using the record below.",
        ),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{question}"),
    ]
)


def load_memory(_):
    return memory.load_memory_variables({})["history"]


chain = RunnablePassthrough.assign(history=load_memory) | final_prompt | llm


def invoke_chain(movie):
    # question = f"What do you know about {movie}?"
    question = movie
    result = chain.invoke({"question": question})
    memory.save_context({"input": question}, {"output": result.content})
    # print(result.content)


# 테스트 실행
invoke_chain("Spider Man")  # 🚀 예상: "Avatar" (첫 질문이므로 영화 제목 출력)      

🕷️🦸‍♂️🕸️

In [46]:
invoke_chain("Avengers")  # 🚀 예상: "Avatar" (첫 질문이므로 영화 제목 출력)

🦸‍♂️🦸‍♀️🌌

In [47]:
invoke_chain("What was the first movie I asked?")  # 예상: "Avatar"

I'm sorry, I cannot recall the previous conversation.

In [44]:
invoke_chain("Why can't you recall the moive name I asked? you are using memory and have the conversation hitory")

I apologize for the confusion. The first movie you asked about was "Spider Man."The human mentions Spider Man and the AI responds with spider and superhero emojis.